In [ ]:
from IPython.display import HTML
from matplotlib_inline.backend_inline import set_matplotlib_formats

# Use inline backend so CSS scaling works on PNG/SVG outputs
set_matplotlib_formats('png')

HTML("""
<style>
/* =======================
   Matplotlib figure sizing
   ======================= */

/* Images (PNG/retina) */
.jp-RenderedImage img,
.jp-OutputArea-output img {
  width: 100% !important;
  height: auto !important;
  max-width: 100% !important;
  display: block;
  box-sizing: border-box;
}

/* SVG outputs */
.jp-RenderedSVG svg,
.jp-OutputArea-output svg {
  width: 100% !important;
  height: auto !important;
}

/* ipympl / canvas-backed figures (in case %matplotlib widget is used) */
.jupyter-matplotlib canvas.mpl-canvas,
.jp-OutputArea-output canvas.mpl-canvas,
.jp-OutputArea-output canvas {
  width: 100% !important;
  height: auto !important;
  max-width: 100% !important;
}

/* Reset container widths */
.jp-OutputArea-output .matplotlib-figure,
.jp-RenderedHTMLCommon .matplotlib-figure,
.jp-OutputArea-output .output_png {
  width: 100% !important;
}

/* =======================
   ipywidgets styling
   ======================= */

/* Make slider labels bold */
.widget-label {
    font-weight: bold;
}

/* Style ipywidgets buttons */
.jp-OutputArea .widget-button,
button.widget-button {
    font-weight: bold;
    background-color: #1976d2;  /* blue background */
    color: white;               /* white text */
    border: none;
    border-radius: 6px;
    padding: 6px 12px;
    font-size: 14px;
    cursor: pointer;
}

/* Button hover effect */
.jp-OutputArea .widget-button:hover,
button.widget-button:hover {
    background-color: #1259a4;  /* darker blue */
}
</style>
""")

# Chirping Away 🐦🎶

## 🎯 Learning goals

- **Understand** how **matched filtering** works to produce a high range resolution from **chirped signals** while maintaining high signal energy.
- **Explore** how **bandwidth** and **SNR** affect **resolution** and detectability.

## 🔭 Resolution, Pulse Duration, and Energy

We've learned that **range resolution** depends on the **pulse duration**. If the pulse is **too long**, echoes from targets that are close to each other can **overlap**, arriving at the receiver **simultaneously**, making it **impossible to distinguish them**. 🚫👀

But there's a catch! A radar transmitter can only output a certain **maximum power**, typically operating **close to that limit**. If we **shorten** the pulse to enhance resolution, we reduce the **total transmitted energy**, because:

$$ \text{Energy} = \text{Power} \times \text{Time} $$

So, when the **pulse duration decreases**, the total **energy also decreases**. 👎

To boost **energy**, we would need to transmit a **longer pulse**. 🔋

To **see far** with our radar, we need:
- ✅ **High energy** in the pulse (more energy = greater detection range).
- ✅ **Minimal noise** in the receiver.

But there's more! Decreasing pulse duration means the receiver must **sample the signal faster**. This higher sampling rate **lets in more noise**, which isn't great for detecting faint signals. 😬

It's all a balancing act between **resolution**, **total energy**, and **noise**! ⚖️

---

## 🚸 Swing Analogy: Short vs. Long Pushes & Total Energy  

**Imagine** you're at a playground, and your friend sits on a swing while you get ready to push.  

1️⃣ **Short, Strong Push** 💨  
   - You give the swing **one quick, forceful push**, sending it up fast.  
   - This corresponds to a **short, high-energy pulse** in radar.  

2️⃣ **Long, Gentle Push** ⏳  
   - Instead of one hard push, you **push gently over a longer time**.  
   - Even though each moment’s push is weaker, the swing **gradually accumulates energy** and reaches the **same final height**.  
   - Translated to radar: a **longer pulse with lower peak power** can achieve the **same detection range**.  

---

## 😕 The Radar Conundrum

⚡ **Maximum Power Limitations** prevent us from transmitting **short, ultra-strong pulses**.  

| **Pulse Type**   | **Pros** ✅ | **Cons** ❌ |  
|------------------|------------|------------|  
| **Short Pulse** ⏳ | High **range resolution**, super sharp targets 🎯 | Low **total energy**, higher **bandwidth**, <br> so more noise 🔊 and limited **range** 📉 |  
| **Long Pulse** 🔋 | High **total energy**, lower **bandwidth**, <br> so less **noise** 🌙 and good for **long range** 🚀 | Poor **range resolution**, targets blend together 📡👀 |  


🤔 **If only** there were a way to get the **high energy** of a long pulse **and** the **fine resolution** of a short pulse... 💡  

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Time axis for plotting
t = np.linspace(0, 5, 500)  # 0 to 5 seconds, with 500 samples

# Define a short, powerful pulse:
# - Occurs from t=1.0 to t=1.5 (duration 0.5 s)
# - Power = 4 units
# => Total energy (area) = power * duration = 4 * 0.5 = 2
short_pulse = np.zeros_like(t)
short_mask = (t >= 1.0) & (t < 1.5)
short_pulse[short_mask] = 4.0

# Define a longer, weaker pulse:
# - Occurs from t=3.0 to t=4.0 (duration 1.0 s)
# - Power = 2 units
# => Total energy (area) = 2 * 1.0 = 2
long_pulse = np.zeros_like(t)
long_mask = (t >= 3.0) & (t < 4.0)
long_pulse[long_mask] = 2.0

# Plot both pulses
fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(t, short_pulse, label='Short, Strong Pulse', color='red')
ax.plot(t, long_pulse, label='Long, Weaker Pulse', color='blue')

# Fill the area under each pulse for clarity
ax.fill_between(t, short_pulse, where=short_mask, color='red', alpha=0.3)
ax.fill_between(t, long_pulse, where=long_mask, color='blue', alpha=0.3)

ax.set_xlabel('Time (s)')
ax.set_ylabel('Power')
ax.set_title('Two Pulses with the Same Total Energy')
ax.legend()
ax.grid(True)

plt.tight_layout()
plt.show()

## 🎹 Piano Scale Analogy: Resolving Overlapping Echoes with Frequency Coding  

Imagine a pianist playing a **rising set of pitches** (a scale) on the piano, like this:  
**A → B → C → D → E → F → G → A**  

At first, the pianist plays this scale **with their right hand** alone, and everything is clear and distinct. But now, let’s add a twist:

---

### 🎼 1. Two Identical, Delayed & Overlapping Scales

Now, the pianist decides to **start playing the exact same scale with their left hand**, but **slightly later** than the right hand.  

- The **right hand** starts the scale by playing the note **A**, then gradually moving up the scale. 🎶 
- The **left hand** starts the same scale from **A** just a few moments later, after the right hand has played the first few notes. 🎶

Since both hands are playing **the same scale**, but offset in time, when you listen to them **together**, the notes overlap! Instead of hearing a single clear melody, you now hear **two notes at the same time**, which blend together. 🎵  

🛑 **The Problem:**  
- To an **untrained ear**, this might sound like **a chaotic jumble of notes**. 🎵❓  
- If you only listen to the **raw sound**, you might **struggle to recognize** that two identical scales are being played.  
- Just like in radar, **overlapping echoes** can initially seem **inseparable**—blending together into a single indistinguishable return. 📡🎶  

---

### 🎵 2. Separating the Overlapping Scales Using Frequency  

Now, let’s think about **what actually happens at any given moment**:  

- The **right-hand scale** is at a higher frequency (higher pitch). ⬆️
- The **left-hand scale** is at a slightly lower frequency (lower pitch). ⬇️  

This means that even though the notes **overlap**, if you listen carefully to the sound at each instant, you can **still resolve the two scales** being played.

📡 **Radar Parallel:**  
- Instead of rising **sound pitches**, a **chirp** in radar is a pulse where the **frequency increases over time**. 🎶➡️📡  
- If two chirp echoes arrive at the receiver with slightly different delays (due to **targets at different distances**), they overlap but their **instantaneous frequencies** will always be **different** at any given moment.  
- This means that by analyzing the **instantaneous frequency** of the received signal, we can **separate received echoes**, even when they mostly **overlap**! 🔍  


---

### 🎶 3. How the Ear (or Radar) Processes the Information  

🎵 **The Perfect Pitch Analogy:**  
Imagine a **musician with perfect pitch** listening to these overlapping scales. 🎼👂  
- Instead of hearing a **jumble of notes**, they focus on the **exact frequency at each moment** and can tell:  
  - “Right now, I hear **a B and a D**, then a moment later I hear **a C and an E**. I must be hearing two separate scales, one slightly ahead of the other in time.” 🎶  
  - By carefully **analyzing the pitches being played at any instant**, they can mentally unravel the two scales, even though the notes **overlap in time**.  

📡 **Radar Parallel:**  
- A **matched filter** in radar processing acts like a trained musician—it "listens" to the **instantaneous frequency** and determines which parts of the signal belong to which echo. 🎯🔊  
- This allows us to **separate overlapping chirp echoes**, just like a musician can recognize two scales being played slightly out of sync.  

---

🎹 **Just as a pianist can separate their two hands’ overlapping scales based on pitch differences, radar can separate overlapping chirp echoes by analyzing frequencies over time!** 🎼📡🎶  

## 🎶 Chirp Waveforms  

A **chirp waveform** is a special type of radar signal where the **frequency increases or decreases over time**. The key advantage of using chirps is that **matched filter processing** can **collect all the pulse energy** and **compress it into a sharp peak** at the correct echo delay. 🐦🎵

This allows us to **combine the high energy of a long pulse** with the **fine resolution of a short pulse**—a powerful technique in radar! **A rare moment of eating one's cake and having it too!** 🎂


### 🖥️ Simulating a Chirp  

Below, you’ll find a code snippet to **generate and visualize a chirp waveform**. Try experimenting with its key parameters:  
- **Pulse duration** (how long the chirp lasts)  
- **Total bandwidth** (the range of frequencies it sweeps through)

From these, the **chirp rate** is determined, which defines **how quickly the frequency changes** within the pulse. Play around with these values and observe how they affect the waveform! 🎵📡  

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def build_chirp_simulation():
    """
    Returns a Voila/Notebook UI for an interactive linear chirp simulation.
    - Sliders: pulse duration (s), bandwidth (Hz)
    - Sampling rate is set to 5 * bandwidth (derived)
    - Plots: time-domain waveform and instantaneous frequency
    Everything is wrapped to avoid interfering with other notebook state.
    """
    import ipywidgets as widgets
    from IPython.display import display, clear_output
    import os
    from datetime import datetime

    # ------------ plotting core (kept local to avoid global namespace pollution) ------------
    def _plot_chirp(pulse_duration, bandwidth):
        # Derived sampling rate and Nyquist guidance
        sampling_rate = 5.0 * bandwidth
        nyquist_min = bandwidth  # For a baseband chirp sweeping ±B/2, fs >= B

        # Time vector
        n = int(max(8, sampling_rate * pulse_duration))  # enforce a minimal sample count
        t = np.linspace(0, pulse_duration, n, endpoint=False)

        # Chirp params (baseband linear chirp from -B/2 to +B/2)
        f0 = -bandwidth / 2
        f1 =  bandwidth / 2
        k = (f1 - f0) / pulse_duration  # chirp rate (Hz/s)

        # Phase and signal
        phase = 2 * np.pi * (f0 * t + 0.5 * k * t**2)
        chirp_signal = np.cos(phase)

        # Instantaneous frequency
        inst_freq = f0 + k * t

        # --- plots ---
        fig, axs = plt.subplots(2, 1, figsize=(9, 6))
        # Time-domain
        axs[0].plot(t * 1e6, chirp_signal, label="Chirp waveform")
        axs[0].set_title("Chirp Signal (Time Domain)")
        axs[0].set_xlabel("Time (µs)")
        axs[0].set_ylabel("Amplitude")
        axs[0].grid(True)
        axs[0].legend(loc="upper right")

        # Instantaneous frequency
        axs[1].plot(t * 1e6, inst_freq * 1e-6, label="Instantaneous Frequency")
        axs[1].set_title("Instantaneous Frequency")
        axs[1].set_xlabel("Time (µs)")
        axs[1].set_ylabel("Frequency (MHz)")
        axs[1].grid(True)
        axs[1].legend(loc="upper left")

        fig.tight_layout()

        return sampling_rate, nyquist_min

    # ------------ widgets ------------
    pulse_slider = widgets.FloatLogSlider(
        value=10e-6, base=10, min=-8, max=-3, step=0.01,
        description='Pulse duration (s):', readout_format='.3e', continuous_update=False
    )
    pulse_slider.style.description_width = 'initial'
    bw_slider = widgets.FloatLogSlider(
        value=10e6, base=10, min=4, max=9, step=0.01,
        description='Bandwidth (Hz):', readout_format='.3e', continuous_update=False
    )
    bw_slider.style.description_width = 'initial'

    info_html = widgets.HTML(
        value="",
        placeholder="info",
        description=""
    )

    out = widgets.Output()

    def _update_info(sr, nyq):
        info_html.value = (
            f"<b>Derived sampling rate:</b> {sr:,.0f} Hz ( = 5 × B )"
            f"<br><b>Nyquist minimum:</b> ≥ {nyq:,.0f} Hz"
        )

    def _redraw(*_):
        with out:
            clear_output(wait=True)
            sr, nyq = _plot_chirp(pulse_slider.value, bw_slider.value)
            _update_info(sr, nyq)
            plt.show()

    # Initial draw and event bindings
    pulse_slider.observe(_redraw, names='value')
    bw_slider.observe(_redraw, names='value')
    _redraw()

    # Add Reset/Save buttons
    reset_btn = widgets.Button(description='Reset')
    save_btn  = widgets.Button(description='Save PNG')

    def _ensure_outputs_dir():
        out_dir = os.path.join('outputs')
        os.makedirs(out_dir, exist_ok=True)
        return out_dir

    def _on_reset(_):
        pulse_slider.value = 10e-6
        bw_slider.value = 10e6
        _redraw()

    def _on_save(_):
        out_dir = _ensure_outputs_dir()
        ts = datetime.now().strftime('%Y%m%d_%H%M%S')
        plt.gcf().savefig(os.path.join(out_dir, f'chirp_waveform_{ts}.png'), dpi=120, bbox_inches='tight')

    reset_btn.on_click(_on_reset)
    save_btn.on_click(_on_save)

    controls = widgets.VBox([
        widgets.HBox([pulse_slider, bw_slider, reset_btn, save_btn]),
        info_html
    ])

    ui = widgets.VBox([controls, out])

    # In Voila, returning the widget is sufficient to render it
    return ui

chirp_simulation = build_chirp_simulation()
display(chirp_simulation)

### 🧪 Try this
- Increase **Bandwidth**; watch instantaneous frequency slope steepen and time plot densify.
- Shorten **Pulse duration**; see fewer cycles but same frequency excursion.

## 🐦🎵 Processing Chirped Pulses: Resolving Overlapping Echoes  

We’ve learned the **physical principle** behind transmitting a **long pulse** to achieve **high energy**, while still maintaining the **resolution of a short pulse**. But how do we actually process the data to get this resolution?  🔍

We can't simply ask the user to analyze the changing frequencies in the received signal like a professional musician listening for overlapping scales 🎶 (even though that is, in essence, the **physical mechanism** separating the echoes). Instead, we need a **systematic approach** to extract the target information.  

---

## 🕵️ Detecting Fingerprints   

Imagine a **forensic investigator** dusting for **fingerprints** on a table. They have a **reference fingerprint** 🖐️ and need to check if it appears anywhere on the surface.  

1️⃣ **Sliding the Reference Print** ⬅️➡️  
   - The investigator **moves** the fingerprint sample across different areas of the table.  

2️⃣ **Comparing Patterns** 🔎  
   - At each position, they **compare** the reference fingerprint with the dust traces left behind.  

3️⃣ **Finding Strong Matches** ✅  
   - Some areas show a **clear match**, revealing distinct fingerprint ridges.  

4️⃣ **Detecting Partial Matches** 🤔  
   - Other areas may have **partial traces**—still a match, but weaker.  

Instead of identifying just **one exact location**, the investigator creates a **map** 🗺️, showing how likely it is that the fingerprint appears at each position.  
- Areas with a **strong match** 🔴 indicate a **high probability** of the fingerprint being present.  
- Areas with **faint traces or no match** ⚪ suggest low or no correlation.  
  
This process is **just like radar signal processing!** Instead of scanning for fingerprints, we **slide a reference signal** across the received data to **detect echoes and measure correlations**. This is what engineers call **matched filtering**! 📡✨  

- The **reference fingerprint** 🖐️ is like the **transmitted chirp signal**.  
- The **dust on the table** represents the **received radar signal**, which may contain **overlapping chirp echoes** from multiple targets.  
- As we **slide the transmitted chirp** over the received echoes, we **measure how much of it is present at each time delay**.  
- **Strong matches** correspond to **targets**, while **weaker matches** indicate **noise or no echoes**.  

To obtain high resolution in the time dealys of the echoes (i.e., in range), we need to **focus** the energy of each chirped pulse. Matched filtering achieves this in a very similar manner to **beam steering**:  

- In **antenna arrays**, signals arrive at different **angles**, creating **phase differences** between receiver elements. By **adjusting the phase shifts** of each element, we can steer the beam and separate overlapping echoes.  
- In **chirp echoes**, signals arrive at different **time delays**, creating a mix of frequencies across time. To separate them, we **adjust the phases** of the received signal so that each echo sums coherently at its correct delay—just like focusing a beam, but in **time** instead of space!  

By **coherently summing** the samples of each echo, we can **compress the energy** into sharp peaks at their correct delays—**revealing target positions with high resolution**. 🎯✨  

## 🎼 Matched Filtering in Action  

In the simulation below, we imagine a radar system that has recorded **chirp echoes** from two targets. 🎯🎯 You can adjust the delay of the chirps, which corresponds to changing the **range of the targets**.  

- In the **first plot**, you see the two chirp echoes separately.  
- But because the targets are close to each other, their reflected chirps **overlap in time** ⏱️.  
- And we know what that means — the waves combine and create an **interference pattern** ⚡, shown in the **second plot**.  

Just by looking at this raw data, we can’t tell the two targets apart — it looks like a **jumbled mess** 😵‍💫.  

But here comes **matched filtering** to the rescue 🦸‍♂️✨:  We slide a reference chirp across the time axis and, at each delay, calculate the **correlation** between the reference and the received echoes. The result — shown in the **third plot** — reveals the two targets as **clearly separated compressed peaks** ⛰️⛰️.  

In [ ]:
def build_chirp_correlation_sim():
    import numpy as np
    import matplotlib.pyplot as plt
    from ipywidgets import FloatSlider, Play, IntSlider as IS, jslink, HBox, VBox, Layout, Output, Label, Button
    from IPython.display import display, clear_output
    import os
    from datetime import datetime

    # -----------------------
    # Fixed parameters (tweak as desired)
    # -----------------------
    chirp_rate     = 2.0   # 1/s^2
    pulse_duration = 4.0   # s
    total_time     = 8.0   # s
    FRAMES         = 150   # timeline length (number of shift samples)
    INTERVAL_MS    = 60    # Play interval (ms)
    DPI            = 96

    # -----------------------
    # Widgets (match your control style)
    # -----------------------
    delay1 = FloatSlider(value=0.00, min=-total_time/2, max=total_time/2, step=0.01,
                         description="Delay 1 (s)", continuous_update=False)
    delay1.style.description_width = 'initial'
    delay2 = FloatSlider(value=0.50, min=-total_time/2, max=total_time/2, step=0.01,
                         description="Delay 2 (s)", continuous_update=False)
    delay2.style.description_width = 'initial'

    play = Play(value=0, min=0, max=FRAMES, step=1, interval=INTERVAL_MS)
    t_slider = IS(value=0, min=0, max=FRAMES, step=1, description="Time")
    jslink((play, 'value'), (t_slider, 'value'))

    recompute_btn = Button(description="Recompute", tooltip="Recompute signals/correlation for current delays")

    readout = Label()
    reset_btn = Button(description='Reset')
    save_btn  = Button(description='Save PNG')

    controls = HBox([delay1, delay2, play, t_slider, recompute_btn, reset_btn, save_btn],
                    layout=Layout(align_items='center', column_gap='12px'))

    # -----------------------
    # Output / Figure
    # -----------------------
    out = Output()
    fig, (ax0, ax1, ax2) = plt.subplots(3, 1, figsize=(8, 11), dpi=DPI)
    fig.subplots_adjust(hspace=0.4)  # <-- add more vertical space
    plt.close(fig)  # prevent static snapshot in notebooks/Voilà

    # -----------------------
    # Local helpers
    # -----------------------
    def rect(t, width=1.0, center=0.0):
        return np.where((t >= center - width/2) & (t <= center + width/2), 1.0, 0.0)

    def chirp_signal(t, cr=1.0):
        # s(t) = exp(j * pi * cr * t^2)
        return np.exp(1j * np.pi * cr * t**2)

    # -----------------------
    # State
    # -----------------------
    state = {
        "t": None,
        "shifts": None,
        "chirp1": None,
        "chirp2": None,
        "rx": None,
        "corr_vals": None,
        "lines": {},  # will hold matplotlib Line2D references
        "cached_delays": (None, None),
        "max_corr": 1.0,
    }

    # -----------------------
    # Build/compute scene (called when delays change)
    # -----------------------
    def _recompute_scene(d1, d2):
        # Time sampling guided by your Nyquist note:
        # Bandwidth B = chirp_rate * pulse_duration
        nyquist_frequency = 2.0 * chirp_rate * pulse_duration

        # Keep sizes modest for Voila responsiveness
        time_samples = int(np.clip(2 * total_time * nyquist_frequency, 512, 2000))
        t = np.linspace(-total_time/2, total_time/2, time_samples, endpoint=True)

        # Shifts sampled across the same window; number of steps = FRAMES
        shifts = np.linspace(-total_time/2, total_time/2, FRAMES+1, endpoint=True)

        # Signals
        chirp1 = chirp_signal(t - d1, chirp_rate) * rect(t, pulse_duration, d1)
        chirp2 = chirp_signal(t - d2, chirp_rate) * rect(t, pulse_duration, d2)
        rx = chirp1 + chirp2

        # Correlation vs shift (simple overlap sum with unwindowed reference)
        corr_vals = np.zeros_like(shifts, dtype=complex)
        ref_template = chirp_signal  # alias for speed
        for i, s in enumerate(shifts):
            ref = ref_template(t - s, chirp_rate)
            overlap = rx * np.conjugate(ref)
            corr_vals[i] = np.sum(overlap)

        state["t"] = t
        state["shifts"] = shifts
        state["chirp1"] = chirp1
        state["chirp2"] = chirp2
        state["rx"] = rx
        state["corr_vals"] = corr_vals
        state["cached_delays"] = (d1, d2)
        state["max_corr"] = max(1e-9, float(np.max(np.abs(corr_vals))))

        _rebuild_axes(d1, d2)

    # -----------------------
    # Axes/lines construction (once per recompute)
    # -----------------------
    def _rebuild_axes(d1, d2):
        t = state["t"]
        chirp1 = state["chirp1"]
        chirp2 = state["chirp2"]
        shifts = state["shifts"]
        rx = state["rx"]
        max_corr = state["max_corr"]

        ax0.cla(); ax1.cla(); ax2.cla()
        # Top: individual chirps (real parts)
        ax0.set_title('Individual Delayed Chirps')
        l_c1, = ax0.plot(t, np.real(chirp1), label=f'Chirp 1 (delay={d1:.2f}s)')
        l_c2, = ax0.plot(t, np.real(chirp2), label=f'Chirp 2 (delay={d2:.2f}s)')
        ax0.set_xlim(t[0], t[-1])
        min_val = min(np.real(chirp1).min(), np.real(chirp2).min(), -1.2)
        max_val = max(np.real(chirp1).max(), np.real(chirp2).max(),  1.2)
        ax0.set_ylim(min_val, max_val)
        ax0.legend(loc='upper right')
        ax0.set_xlabel('Time (s)'); ax0.set_ylabel('Amplitude')

        # Middle: received + sliding reference (reference updated per frame)
        ax1.set_title('Received Signal (Sum of Delayed Chirps)')
        l_rx, = ax1.plot(t, np.real(rx), label='Received echo signal')
        l_ref, = ax1.plot([], [], linestyle='--', label='Shifted reference chirp')
        ax1.axvline(d1, linestyle='--', label=f'Delay 1 = {d1:.2f}s')
        ax1.axvline(d2, linestyle='--', label=f'Delay 2 = {d2:.2f}s')
        ax1.legend(loc='upper right')
        ax1.set_xlim(t[0], t[-1])
        ax1.set_xlabel('Time (s)'); ax1.set_ylabel('Amplitude')

        # Bottom: correlation magnitude (grows with time slider)
        ax2.set_title('Correlation Output (Magnitude)')
        ax2.axvline(d1, linestyle='--', label=f'Delay 1 = {d1:.2f}s')
        ax2.axvline(d2, linestyle='--', label=f'Delay 2 = {d2:.2f}s')
        l_corr, = ax2.plot([], [], label='|Correlation|')
        ax2.legend(loc='upper right')
        ax2.set_xlim(shifts[0], shifts[-1])
        ax2.set_ylim(0, 1.1 * max_corr)
        ax2.set_xlabel('Shift (s)'); ax2.set_ylabel('Magnitude')

        state["lines"] = {
            "ref": l_ref,
            "corr": l_corr,
            "rx": l_rx,
            "c1": l_c1,
            "c2": l_c2
        }

    # -----------------------
    # Frame draw (called on time slider / play)
    # -----------------------
    def _draw(frame_idx: int):
        # Bound check
        frame_idx = int(np.clip(frame_idx, 0, FRAMES))
        t = state["t"]; shifts = state["shifts"]
        l_ref = state["lines"]["ref"]
        l_corr = state["lines"]["corr"]

        # Current shift for this frame
        s = shifts[frame_idx]

        # Update shifted reference (windowed for visualization consistency)
        ref = chirp_signal(t - s, chirp_rate) * rect(t, pulse_duration, s)
        l_ref.set_data(t, np.real(ref))

        # Update correlation shown up to this frame
        l_corr.set_data(shifts[:frame_idx+1], np.abs(state["corr_vals"][:frame_idx+1]))

        # Readout
        d1, d2 = state["cached_delays"]
        readout.value = (f"chirp_rate = {chirp_rate:.2f} s⁻² | T_p = {pulse_duration:.2f} s | "
                         f"Total time = {total_time:.2f} s | Delay1 = {d1:.2f} s | Delay2 = {d2:.2f} s | "
                         f"Shift = {s:+.2f} s")

        with out:
            clear_output(wait=True)
            display(fig)

    # -----------------------
    # Wire up interactions
    # -----------------------
    def _on_recompute(_=None):
        _recompute_scene(delay1.value, delay2.value)
        _draw(t_slider.value)

    def _on_delay_change(_):
        _on_recompute()

    def _on_time_change(change):
        _draw(change["new"])

    # Observe controls
    delay1.observe(_on_delay_change, names='value')
    delay2.observe(_on_delay_change, names='value')
    t_slider.observe(_on_time_change, names='value')
    recompute_btn.on_click(_on_recompute)

    def _ensure_outputs_dir():
        out_dir = os.path.join('outputs')
        os.makedirs(out_dir, exist_ok=True)
        return out_dir

    def _on_reset(_):
        delay1.value = 0.00
        delay2.value = 0.50
        t_slider.value = 0
        _on_recompute()

    def _on_save(_):
        out_dir = _ensure_outputs_dir()
        ts = datetime.now().strftime('%Y%m%d_%H%M%S')
        fig.savefig(os.path.join(out_dir, f'chirp_correlation_{ts}.png'), dpi=120, bbox_inches='tight')

    reset_btn.on_click(_on_reset)
    save_btn.on_click(_on_save)

    # Initial compute + draw
    _on_recompute()

    return VBox([controls, readout, out])

# Example usage:
correlation_simulation = build_chirp_correlation_sim()
display(correlation_simulation)

### 🧪 Try this
- Move **Delays 1 and 2** closer until chirps heavily overlap; watch the raw/interference views.
- Press **Recompute** to refresh correlation; observe two peaks separate when delays differ.

## ⚡ Range Compression: Sharpening Radar Echoes  

The process of **calculating the correlation** of the received echo signal with the transmitted chirp (i.e., matched filtering) is also called **range compression** in radar lingo. 🗜️ Why? Because it **"compresses"** the energy of a long pulse into a **narrow, high peak**, sharpening the radar’s ability to detect targets with precision. 🎯  

---

### 📏 What Determines Range Resolution?  

The **resolution** is defined by **the width of the compressed peak**. And what determines this width? **Surprise, surprise—it's the bandwidth!** 📻  

- The **wider** the bandwidth (the difference between maximum and minimum frequency), the **narrower** the compressed peak.  
- This means we achieve **the same resolution** that we would have gotten if we had transmitted an extremely short pulse, corresponding to the width of the correlation peak, but without the chirp modulation! 

---

### 🪄 The Magic of Pulse Compression  

Here’s the real **game-changer**:  Matched filtering **collects the entire energy of the chirped pulse into a sharp peak**.  

✨ **It’s as if we transmitted an extremely powerful, short pulse!** ✨  

But instead of requiring **massive instantaneous transmit power**, we achieve this entirely **digitally**—by processing the recorded waves in a computer. **That’s real radar magic!** 🔮  

---

### 🔊 Boosting SNR: Why Pulse Compression Helps in Noise  

One of the **coolest** advantages of collecting the **entire pulse energy** into a peak is that it **improves the signal-to-noise ratio (SNR)**.  

Imagine that the **chirp is buried in strong background noise**. What happens when we perform the **matched filtering** via cross-correlation?  

✅ **At the correct target delay**, the wave samples from the entire chirp **align in phase** and sum together **coherently**, reinforcing the signal.  
✅ **The noise, however, is random**—its phases do not align, so it does **not sum coherently** like the signal does.  
✅ As a result, **the signal energy increases**, but the **noise remains mostly unchanged**, effectively **boosting the SNR!** 🎯  

---

### 🛠️ Play with Bandwidth & SNR  

The code snippet below lets you **explore** two fundamental concepts:  

🔹 **Range Compression & Resolution** 🎯  
   - Increase the **bandwidth** and watch as the **compressed peaks sharpen**, allowing you to **resolve closely spaced targets** with greater ability.  

🔹 **SNR Improvement** 🔊➡️📈  
   - Lower the **SNR** to **bury the echoes in noise**—then see how **matched filtering can still reveal the targets!**  
   - Even when the **raw echo signal disappears into noise**, the **matched filter output** brings the targets **back into view**! 👀✨  

In [ ]:
def build_resolution_snr_static_sim():
    import numpy as np
    import matplotlib.pyplot as plt
    from ipywidgets import (
        FloatSlider, FloatLogSlider, HBox, VBox, Layout, Output, Label, Button
    )
    from IPython.display import display, clear_output
    import os
    from datetime import datetime

    # -----------------------
    # Constants & settings
    # -----------------------
    c   = 3e8    # speed of light (m/s)
    DPI = 96
    COL_T1 = 'tab:blue'   # Target 1 color
    COL_T2 = 'tab:orange' # Target 2 color

    # -----------------------
    # Widgets (independent B and T_p)
    # -----------------------
    bandwidth = FloatLogSlider(
        value=1e6, base=10, min=4, max=8, step=0.01,
        description="Bandwidth B [Hz]", continuous_update=False, readout_format=".3e"
    )
    pulse_dur = FloatLogSlider(
        value=2e-5, base=10, min=-6, max=-3, step=0.01,
        description="Pulse Tₚ [s]", continuous_update=False, readout_format=".3e"
    )
    snr_db = FloatSlider(
        value=10.0, min=-40.0, max=40.0, step=1.0,
        description="SNR [dB]", continuous_update=False
    )
    r1 = FloatSlider(
        value=2000.0, min=0.0, max=10000.0, step=10.0,
        description="Target 1 [m]", continuous_update=False
    )
    r2 = FloatSlider(
        value=2100.0, min=0.0, max=10000.0, step=10.0,
        description="Target 2 [m]", continuous_update=False
    )
    recompute_btn = Button(description="Recompute", tooltip="Recompute with current parameters")
    readout = Label()

    reset_btn = Button(description='Reset')
    save_btn  = Button(description='Save PNG')

    controls_top = HBox([bandwidth, pulse_dur, snr_db],
                        layout=Layout(align_items='center', column_gap='12px'))
    controls_bot = HBox([r1, r2, recompute_btn, reset_btn, save_btn],
                        layout=Layout(align_items='center', column_gap='12px'))

    # -----------------------
    # Output / Figure (extra spacing)
    # -----------------------
    out = Output()
    fig, (ax0, ax1, ax2) = plt.subplots(3, 1, figsize=(8, 11), dpi=DPI)
    fig.subplots_adjust(hspace=0.5)
    plt.close(fig)  # avoid static snapshot in Voilá

    # -----------------------
    # Helpers
    # -----------------------
    def rect(t, width=1.0, center=0.0):
        return np.where((t >= center - width/2) & (t <= center + width/2), 1.0, 0.0)

    def chirp_signal(t, k=1.0):
        # Baseband linear FM (quadratic phase): s(t) = exp(j*pi*k*t^2)
        return np.exp(1j * np.pi * k * t**2)

    # -----------------------
    # Main recompute & draw
    # -----------------------
    def _recompute_and_draw(*_):
        B  = float(bandwidth.value)
        Tp = float(pulse_dur.value)
        SNRdB = float(snr_db.value)
        R1 = float(r1.value)  # Keep 1:1 mapping to sliders
        R2 = float(r2.value)

        # Derived quantities
        TBP = B * Tp                   # time-bandwidth product
        k   = B / Tp if Tp > 0 else 0 # chirp rate (Hz/s)
        res = c / (2 * B)             # theoretical range resolution (m)
        tau1 = 2 * R1 / c             # delays
        tau2 = 2 * R2 / c

        # Time support to include both echoes
        total_time = max(tau1, tau2) + Tp

        # Sampling (complex baseband): Nyquist ~ 2B; 2× oversampling
        nyquist_frequency = 2.0 * B
        time_samples = int(np.clip(2 * total_time * nyquist_frequency, 1024, 20000))
        t = np.linspace(0.0, total_time, time_samples, endpoint=True)
        dt = t[1] - t[0]
        range_axis = (c * t) / 2.0

        # Build echoes (Target 1 = blue, Target 2 = orange)
        s1 = chirp_signal(t - tau1, k) * rect(t, Tp, tau1)
        s2 = chirp_signal(t - tau2, k) * rect(t, Tp, tau2)
        rx = s1 + s2

        # Add complex AWGN for given SNR
        signal_power = np.mean(np.abs(rx)**2)
        noise_power  = signal_power / (10.0**(SNRdB / 10.0))
        noise = np.sqrt(noise_power/2.0) * (np.random.randn(time_samples) + 1j*np.random.randn(time_samples))
        rx_noisy = rx + noise

        # Cross‑correlation vs shift (full curve)
        num_shifts = int(np.clip(2 * total_time * nyquist_frequency, 1024, 16000))
        shifts = np.linspace(0.0, total_time, num_shifts, endpoint=True)
        range_shifts = (c * shifts) / 2.0
        corr_values = np.zeros(num_shifts, dtype=complex)
        for i, s in enumerate(shifts):
            ref = chirp_signal(t - s, k) * rect(t, Tp, s)
            corr_values[i] = np.sum(rx_noisy * np.conjugate(ref))

        # ----------------- Plot -----------------
        ax0.cla(); ax1.cla(); ax2.cla()

        # 1) Individual target echoes (real part) vs range
        ax0.set_title('Individual Target Echoes (Real Part)')
        ax0.plot(range_axis, np.real(s1), color=COL_T1, label=f'Target 1 (R={R1:.1f} m)')
        ax0.plot(range_axis, np.real(s2), color=COL_T2, label=f'Target 2 (R={R2:.1f} m)')
        ax0.set_xlim(range_axis[0], range_axis[-1])
        ax0.set_ylim(-1.2, 1.2)
        ax0.set_xlabel('Range (m)'); ax0.set_ylabel('Amplitude')
        ax0.legend(loc='upper right')

        # 2) Received noisy signal (real part) vs range
        ax1.set_title(f'Received Signal with Noise (SNR = {SNRdB:.0f} dB)')
        ax1.plot(range_axis, np.real(rx_noisy), color='tab:gray', label='Noisy Received Signal')
        ax1.axvline(R1, color=COL_T1, linestyle='--', label=f'Target 1 = {R1:.1f} m')
        ax1.axvline(R2, color=COL_T2, linestyle='--', label=f'Target 2 = {R2:.1f} m')
        ax1.set_xlim(range_axis[0], range_axis[-1])
        ax1.set_xlabel('Range (m)'); ax1.set_ylabel('Amplitude')
        ax1.legend(loc='upper right')

        # 3) Correlation magnitude vs range
        ax2.set_title('Cross-Correlation Output (Magnitude) in Range')
        ax2.axvline(R1, color=COL_T1, linestyle='--', label=f'Target 1 = {R1:.1f} m')
        ax2.axvline(R2, color=COL_T2, linestyle='--', label=f'Target 2 = {R2:.1f} m')
        ax2.plot(range_shifts, np.abs(corr_values), color='tab:green', label='|Correlation|')
        ax2.set_xlim(range_shifts[0], range_shifts[-1])
        ax2.set_ylim(0, 1.1 * np.max(np.abs(corr_values)))
        ax2.set_xlabel('Range (m)'); ax2.set_ylabel('Magnitude')
        ax2.legend(loc='upper right')

        # Readout
        readout.value = (
            f"B = {B:.3e} Hz | Tₚ = {Tp:.3e} s | TBP = B·Tₚ = {B*Tp:.2f} | "
            f"ΔR = c/(2B) = {c/(2*B):.2f} m | "
            f"R₁ = {R1:.1f} m, R₂ = {R2:.1f} m | dt = {dt:.3e} s"
        )

        with out:
            clear_output(wait=True)
            display(fig)

    # Hook up events
    for w in (bandwidth, pulse_dur, snr_db, r1, r2):
        w.observe(_recompute_and_draw, names='value')
    recompute_btn.on_click(_recompute_and_draw)

    def _ensure_outputs_dir():
        out_dir = os.path.join('outputs')
        os.makedirs(out_dir, exist_ok=True)
        return out_dir

    def _on_reset(_):
        bandwidth.value = 1e6
        pulse_dur.value = 2e-5
        snr_db.value = 10.0
        r1.value = 2000.0
        r2.value = 2100.0
        _recompute_and_draw()

    def _on_save(_):
        out_dir = _ensure_outputs_dir()
        ts = datetime.now().strftime('%Y%m%d_%H%M%S')
        fig.savefig(os.path.join(out_dir, f'resolution_snr_{ts}.png'), dpi=120, bbox_inches='tight')

    reset_btn.on_click(_on_reset)
    save_btn.on_click(_on_save)

    # Initial render
    _recompute_and_draw()

    return VBox([controls_top, controls_bot, readout, out])

# Example:
ui = build_resolution_snr_static_sim()
display(ui)

### 🧪 Try this
- Increase **Bandwidth**; peaks narrow and close targets separate better (range resolution ΔR = c/(2B)).
- Lower **SNR**; note how correlation still reveals peaks even when the raw echo looks noisy.
- Bring **Targets 1 and 2** closer until peaks merge; increase B and watch them separate again.


## 📌 Summary

- **Chirps** deliver high **energy** and fine **resolution** when paired with **matched filtering**.
- **Range compression** concentrates pulse energy into **sharp peaks** at true time delays.
- **Bandwidth** sets resolution; matched filtering increases **SNR** and boosts detectability.
